In [1]:
%pwd

'/home/j-i13a103'

In [19]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [66]:
# 커스텀
def custom_formatter(root_path, manifest_file, **kwargs):
    """
    Assumes each line as ```<filename>|<transcription>```
    """
    items = []

    with open(os.path.join(root_path, manifest_file), "r", encoding="utf-8") as f:
        for line in f:
            cols = line.strip().split("|")
            audio_file = os.path.join(root_path, "wavs", cols[0])
            text = cols[1].strip()
            speaker_name = cols[-1].strip()
            items.append({
                "text": text,
                "audio_file": audio_file,
                "speaker_name": speaker_name,
                "language": "ko",
                "root_path": root_path
            })

    return items

In [36]:
from TTS.tts.datasets.formatters import register_formatter

register_formatter("custom_formatter", custom_formatter)

In [67]:
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples

root_path = "tts/MyTTSDataset/"

# Define here the dataset that you want to use for the fine-tuning on.
dataset_config = BaseDatasetConfig(
    formatter="custom_formatter",
    meta_file_train="metadata_with_emotion_tag_train.txt",
    meta_file_val="metadata_with_emotion_tag_val.txt",
    path=root_path,    # dataset 루트 경로
    language="ko"
)

# train_samples, eval_samples = load_tts_samples(
#     dataset_config,
#     eval_split=True,
#     eval_split_size=0.1,
#     formatter=custom_formatter    # 커스텀 formatter 함수 전달하기
# )

In [68]:
# add here the configs of the datasets
DATASETS_CONFIG_LIST = [dataset_config]

In [ ]:
# print(len(train_samples), len(eval_samples))

In [39]:
# 사용할 모델 : tts_models/multilingual/multi-dataset/xtts_v2
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig
from TTS.utils.manage import ModelManager

# 로깅
RUN_NAME = "GPT_XTTS_v2.0_MyDataset_FT"
PROJECT_NAME="XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

OUT_PATH = os.path.join(os.path.dirname(os.path.abspath(root_path)), "run", "training")
print(OUT_PATH)

/home/j-i13a103/tts/run/training


In [52]:
# Training 파라미터
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True
START_WITH_EVAL = True
BATCH_SIZE = 3
GRAD_ACUMM_STEPS = 84

# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training.
# You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly

In [41]:
# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")

os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

In [24]:
# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

print(DVAE_CHECKPOINT)
print(MEL_NORM_FILE)

/home/j-i13a103/tts/run/training/XTTS_v2.0_original_model_files/dvae.pth
/home/j-i13a103/tts/run/training/XTTS_v2.0_original_model_files/mel_stats.pth


In [ ]:
# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


 > Downloading DVAE files!


100%|██████████| 1.07k/1.07k [00:00<00:00, 1.31kiB/s]


In [26]:
# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

print(TOKENIZER_FILE)
print(XTTS_CHECKPOINT)

/home/j-i13a103/tts/run/training/XTTS_v2.0_original_model_files/vocab.json
/home/j-i13a103/tts/run/training/XTTS_v2.0_original_model_files/model.pth


In [27]:
# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )

 > Downloading XTTS v2.0 files!


100%|██████████| 211M/211M [00:50<00:00, 4.14MiB/s]
361kiB [00:00, 593kiB/s]


In [28]:
# Training sentences generations
SPEAKER_REFERENCE = [
    "./tts/speaker_reference.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = dataset_config.language

In [77]:
from trainer import Trainer, TrainerArgs

from TTS.tts.configs.xtts_config import XttsAudioConfig

def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6 secs
        min_conditioning_length=22050,  # 1 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
    
    # training parameters config
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        eval_split_size=0.1,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        
        # target_loss="loss",
        print_eval=False,
        
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "이 케이크 정말 좋네요. 촉촉하고 달콤해서 입에서 사르르 녹아요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "조용한 밤에 들리는 바람 소리는 참 정겨워요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        formatter=custom_formatter
    )

    print(len(train_samples), len(eval_samples))

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )

    trainer.fit()

In [76]:
if __name__ == "__main__":
    main()

549431 61052


fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 80
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/home/j-i13a103/tts/run/training/GPT_XTTS_v2.0_MyDataset_FT-July-28-2025_07+54AM-0000000

 > Model has 518442047 parameters

 > EPOCH: 0/999
 --> /home/j-i13a103/tts/run/training/GPT_XTTS_v2.0_MyDataset_FT-July-28-2025_07+54AM-0000000
 ! Run is removed from /home/j-i13a103/tts/run/training/GPT_XTTS_v2.0_MyDataset_FT-July-28-2025_07+54AM-000

SystemExit: 1

/opt/tljh/user/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
